<a href="https://colab.research.google.com/github/Alirs2002/tweet_sentiment_analysis/blob/main/code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

in this motherfucker project, i am going to create a model by keras,to classify tweets,to 2 classes

1-disaster

2-normal

lets import necessary libraries

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

now,lets download our dataset

In [2]:
!wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"

--2023-10-05 20:02:28--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.188.207, 173.194.174.207, 74.125.23.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.188.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K   845KB/s    in 0.7s    

2023-10-05 20:02:29 (845 KB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [3]:
!ls

nlp_getting_started.zip  sample_data


In [4]:
!ls ..

bin			    etc     media		      root  tools
boot			    home    mnt			      run   usr
content			    lib     NGC-DL-CONTAINER-LICENSE  sbin  var
cuda-keyring_1.0-1_all.deb  lib32   opt			      srv
datalab			    lib64   proc		      sys
dev			    libx32  python-apt		      tmp


now,lets create a function to unzip our data

In [5]:
import zipfile

def unzip_data(path):

  zip_file_ref = zipfile.ZipFile(path,"r")

  zip_file_ref.extractall()

  zip_file_ref.close()

In [6]:
zip_file_path = "./nlp_getting_started.zip"

In [7]:
unzip_data(zip_file_path)

In [8]:
!ls

nlp_getting_started.zip  sample_submission.csv	train.csv
sample_data		 test.csv


now,lets import our train and test datasets

In [9]:
train_data_path = "./train.csv"

test_data_path = "./test.csv"

In [10]:
train_data = pd.read_csv(train_data_path)

test_data = pd.read_csv(test_data_path)

In [11]:
train_data

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [12]:
pd.options.display.max_colwidth=200

In [13]:
train_data[:2][["text","target"]]

,text,target
0,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,1
1,Forest fire near La Ronge Sask. Canada,1


In [14]:
train_data["target"].value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [15]:
train_data.tail()

,id,keyword,location,text,target
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse into nearby homes http://t.co/STfMbbZFB5,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control wild fires in California even in the Northern part of the state. Very troubling.,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. http://t.co/zDtoyd8EbJ,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided with a car in Little Portugal. E-bike rider suffered serious non-life threatening injuries.,1
7612,10873,NaN,NaN,The Latest: More Homes Razed by Northern California Wildfire - ABC News http://t.co/YmY4rSkQ3d,1


In [16]:
print(len(train_data))

7613


In [17]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [18]:
pd.options.display.max_dir_items=200

In [19]:
train_data["keyword"].isnull().value_counts()

False    7552
True       61
Name: keyword, dtype: int64

In [20]:
train_data[100:105]

,id,keyword,location,text,target
100,144,accident,UK,.@NorwayMFA #Bahrain police had previously died in a road accident they were not killed by explosion https://t.co/gFJfgTodad,1
101,145,accident,"Nairobi, Kenya",I still have not heard Church Leaders of Kenya coming forward to comment on the accident issue and disciplinary measures#ArrestPastorNganga,0
102,146,aftershock,Instagram - @heyimginog,@afterShock_DeLo scuf ps live and the game... cya,0
103,149,aftershock,304,'The man who can drive himself further once the effort gets painful is the man who will win.' \nRoger Bannister,0
104,151,aftershock,Switzerland,320 [IR] ICEMOON [AFTERSHOCK] | http://t.co/yNXnvVKCDA | @djicemoon | #Dubstep #TrapMusic #DnB #EDM #Dance #IcesÛ_ http://t.co/weQPesENku,0


i think ,location and keyword,doesnt have any influenct on our prediction,so we will remove them from our dataset in future

In [21]:
from sklearn.model_selection import train_test_split

In [22]:
train_data = train_data.sample(frac=1,random_state=85)

In [23]:
train_data

,id,keyword,location,text,target
6039,8633,seismic,"Mogadishu, Somalia",Exploration Takes Seismic Shift in #Gabon to #Somalia\nhttp://t.co/Ltf6jL5keU http://t.co/Zlq8tHcTkW,1
1709,2465,collided,San Francisco,Mind blown by @GlassAnimals slithering viscous Gold Mine (cover of Yeah Yeah Yeahs + Erykah Badu)Û_ http://t.co/7Zb9gm5z0h,0
2775,3992,devastation,NaN,70 Years After Atomic Bombs Japan Still Struggles With War Past: The anniversary of the devastation wrought b... http://t.co/vFCtrzaOk2,1
2511,3607,desolation,"North East, England",Just came back from camping and returned with a new song which gets recorded tomorrow. Can't wait! #Desolation #TheConspiracyTheory #NewEP,1
5234,7480,obliteration,NaN,Dead Space - Obliteration Imminent [2/2]: http://t.co/XJB0dCAaHf via @YouTube,0
...,...,...,...,...,...
7053,10106,typhoon,Dhaka,Obama Declares Disaster for Typhoon-Devastated Saipan: Obama signs disaster declaration for Northern Marians a... http://t.co/lEYJwNnAH8,1
1587,2292,cliff%20fall,"Lagos, Nigeria",Huge cliff landslide on road in China: Watch the moment a cliff collapses as huge chunks of rock fall onto a r... http://t.co/eEEwO207mX,1
5828,8321,rubble,Spain - China - Latin America.,China's Stock Market Crash: Are There Gems In The Rubble?: ChinaÛªs stock market crash this summer h... http://t.co/pE2R3lN16o by .Forbes,0
5280,7545,outbreak,Chile,Families to sue over Legionnaires: More than 40 families affected by the fatal outbreak of Legionnaires' disea... http://t.co/02ELqLOpFk,1


In [24]:
train_sentence,val_sentence,train_labels,val_labels = train_test_split(train_data["text"].to_numpy(),
                                                                       train_data["target"].to_numpy(),
                                                                       test_size=0.1,
                                                                       random_state=85)

In [25]:
len(train_sentence),len(train_labels),len(val_sentence),len(val_labels)

(6851, 6851, 762, 762)

In [26]:
train_sentence

array(['Insane bush fires in California. Be safe. https://t.co/jSlxTQ3NqS',
       '@beckyfeigin I defs will when it stops bleeding!',
       "I don't laugh out loud at many things. But man I really lol @ the big bang theory.",
       ...,
       'Only one man Tsutomi Yamaguchi is said to have survived both atomic bomb blasts at #Hiroshima and #Nagasaki. #OTD http://t.co/DaalPeNZp0',
       'Blue Bell May Be Close to a Return From Its Listeria Disaster... Hot on #theneeds #Recipes http://t.co/F56v61AmPt',
       '9:35 pm. Thunderstorm. No rain. 90 degrees. This weather weird.'],
      dtype=object)

now,we have created our training sentence

In [27]:
import random
def show_random(dataset1,dataset2):

  if(len(dataset1)!=len(dataset2)):
    return "fuck you,your fucking datasets are wrong"

  length = len(dataset1)

  random_num = random.randint(0,length)

  random_text = dataset1[random_num]
  random_label = dataset2[random_num]

  if(random_label ==1):
    random_label = "disaster"
  else:
    random_label = "normal"

  print(f"text: {random_text} \n label :{random_label}")

In [28]:
show_random(train_sentence,train_labels)

text: @USAgov Koreans are performing hijacking of the Tokyo Olympic Games.https://t.co/APkSnpLXZj 
 label :disaster


ok,now we should do our next step which is converting our dataset to number

In [29]:
from tensorflow.keras.layers import TextVectorization

In [30]:
vocab_size = 20000

vectorizer = TextVectorization(max_tokens=vocab_size,
                               split="whitespace",
                               standardize="lower_and_strip_punctuation",
                               output_mode="int",
                               output_sequence_length=25)

now,we should adapt our vectorizer

In [31]:
vectorizer.adapt(train_sentence)

In [32]:
vectorizer.vocabulary_size()

20000

In [39]:
len(vectorizer.get_vocabulary())

20000

In [40]:
vectorizer.get_vocabulary()[:10]

['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is']

all right,we have made our vocabulary

lets test it on a simple sentence

In [41]:
simple_sentence = "hello,did you know that zaza pachulia is the greatest player ever?"

simple_matrix = vectorizer(simple_sentence)

simple_matrix

<tf.Tensor: shape=(25,), dtype=int64, numpy=
array([   1,   12,   92,   16,    1,    1,    9,    2, 2090, 2009,  224,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0])>

In [54]:
vectorizer.get_vocabulary()[9999]

'panik'

In [48]:
print(vectorizer("hello"))

tf.Tensor(
[1849    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0], shape=(25,), dtype=int64)


now,lets create a word embedding layer

In [57]:
tf.random.set_seed(85)
from tensorflow.keras import layers

In [59]:
embedding_layer = layers.Embedding(output_dim = 128,
                                   embeddings_initializer="uniform",
                                   name="kir_khar",
                                   input_dim=vocab_size,
                                   )

In [61]:
type(embedding_layer)

keras.src.layers.core.embedding.Embedding

In [72]:
simple_embedding = embedding_layer(vectorizer(simple_sentence))
simple_embedding.shape,simple_embedding

(TensorShape([25, 128]),
 <tf.Tensor: shape=(25, 128), dtype=float32, numpy=
 array([[-0.00842403,  0.03800745, -0.04766575, ...,  0.02209599,
          0.00516837, -0.03507459],
        [-0.04165073,  0.00496406,  0.02717951, ...,  0.00843067,
          0.04539683,  0.04096476],
        [-0.03647285, -0.00416384, -0.0225819 , ...,  0.01552783,
          0.00511833, -0.00051161],
        ...,
        [-0.03554859, -0.00988861, -0.01774279, ..., -0.02480961,
          0.03645251, -0.02219796],
        [-0.03554859, -0.00988861, -0.01774279, ..., -0.02480961,
          0.03645251, -0.02219796],
        [-0.03554859, -0.00988861, -0.01774279, ..., -0.02480961,
          0.03645251, -0.02219796]], dtype=float32)>)

now,lets finally create our fucking model

In [74]:

input_layer = tf.keras.layers.Input(shape=(1,),dtype="string")
x= vectorizer(input_layer)
x= embedding_layer(x)
x = tf.keras.layers.GlobalAveragePooling1D()(x)
x=tf.keras.layers.Dense(1,activation=tf.keras.activations.sigmoid)(x)

model_1 = tf.keras.Model(input_layer,x)


now,lets compile our model

In [75]:
model_1.compile(loss=tf.keras.losses.binary_crossentropy,
                optimizer = tf.keras.optimizers.Adam(learning_rate=0.001),
                metrics=["accuracy"])

In [77]:
model_1_history = model_1.fit(train_sentence,
                               train_labels,
                               epochs=20,
                               validation_data=(val_sentence,val_labels))

Epoch 1/20
215/215 [==============================] - 12s 53ms/step - loss: 0.6367 - accuracy: 0.6490 - val_loss: 0.5605 - val_accuracy: 0.7690
Epoch 2/20
215/215 [==============================] - 12s 55ms/step - loss: 0.4733 - accuracy: 0.8148 - val_loss: 0.4654 - val_accuracy: 0.7874
Epoch 3/20
215/215 [==============================] - 10s 47ms/step - loss: 0.3554 - accuracy: 0.8699 - val_loss: 0.4295 - val_accuracy: 0.8018
Epoch 4/20
215/215 [==============================] - 10s 48ms/step - loss: 0.2748 - accuracy: 0.9024 - val_loss: 0.4252 - val_accuracy: 0.8058
Epoch 5/20
215/215 [==============================] - 10s 45ms/step - loss: 0.2144 - accuracy: 0.9296 - val_loss: 0.4251 - val_accuracy: 0.8176
Epoch 6/20
215/215 [==============================] - 10s 44ms/step - loss: 0.1687 - accuracy: 0.9461 - val_loss: 0.4388 - val_accuracy: 0.8189
Epoch 7/20
215/215 [==============================] - 10s 47ms/step - loss: 0.1333 - accuracy: 0.9587 - val_loss: 0.4537 - val_accuracy:

it looke like,5 epochs was enough and we just increased our overfit

so lets evaluate on our test dataset

so,lets first create our dataset

In [81]:
train_data

,id,keyword,location,text,target
6039,8633,seismic,"Mogadishu, Somalia",Exploration Takes Seismic Shift in #Gabon to #Somalia\nhttp://t.co/Ltf6jL5keU http://t.co/Zlq8tHcTkW,1
1709,2465,collided,San Francisco,Mind blown by @GlassAnimals slithering viscous Gold Mine (cover of Yeah Yeah Yeahs + Erykah Badu)Û_ http://t.co/7Zb9gm5z0h,0
2775,3992,devastation,NaN,70 Years After Atomic Bombs Japan Still Struggles With War Past: The anniversary of the devastation wrought b... http://t.co/vFCtrzaOk2,1
2511,3607,desolation,"North East, England",Just came back from camping and returned with a new song which gets recorded tomorrow. Can't wait! #Desolation #TheConspiracyTheory #NewEP,1
5234,7480,obliteration,NaN,Dead Space - Obliteration Imminent [2/2]: http://t.co/XJB0dCAaHf via @YouTube,0
...,...,...,...,...,...
7053,10106,typhoon,Dhaka,Obama Declares Disaster for Typhoon-Devastated Saipan: Obama signs disaster declaration for Northern Marians a... http://t.co/lEYJwNnAH8,1
1587,2292,cliff%20fall,"Lagos, Nigeria",Huge cliff landslide on road in China: Watch the moment a cliff collapses as huge chunks of rock fall onto a r... http://t.co/eEEwO207mX,1
5828,8321,rubble,Spain - China - Latin America.,China's Stock Market Crash: Are There Gems In The Rubble?: ChinaÛªs stock market crash this summer h... http://t.co/pE2R3lN16o by .Forbes,0
5280,7545,outbreak,Chile,Families to sue over Legionnaires: More than 40 families affected by the fatal outbreak of Legionnaires' disea... http://t.co/02ELqLOpFk,1


In [80]:
test_sentence = test_data["text"]
test_labels = test_data["target"]

KeyError: ignored